# EDA

In [2]:
import pandas as pd

In [3]:
# Import the dataset
dataset_path= "C:/Users/Richie MOUHOUADI/Documents/DSTI/ML with Python/Mental_Health_State_Prediction/Data/Raw_data/Dataset_Patient_Characteristics_Survey__PCS___2019.csv"

dataset = pd.read_csv(dataset_path, sep=";")
dataset.head()

,Survey Year,Program Category,Region Served,Age Group,Sex,Transgender,Sexual Orientation,Hispanic Ethnicity,Race,Living Situation,...,No Insurance,Unknown Insurance Coverage,Medicaid Insurance,Medicaid Managed Insurance,Medicare Insurance,Private Insurance,Child Health Plus Insurance,Other Insurance,Criminal Justice Status,Three Digit Residence Zip Code
0,2019,OUTPATIENT,LONG ISLAND REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"YES, HISPANIC/LATINO",WHITE ONLY,PRIVATE RESIDENCE,...,NO,False,YES,YES,NO,NO,NO,NO,NO,117
1,2019,OUTPATIENT,NEW YORK CITY REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,PRIVATE RESIDENCE,...,NO,False,YES,YES,NO,NO,NO,NO,NO,100
2,2019,OUTPATIENT,HUDSON RIVER REGION,ADULT,MALE,"NO, NOT TRANSGENDER",LESBIAN OR GAY,"NO, NOT HISPANIC/LATINO",WHITE ONLY,PRIVATE RESIDENCE,...,NO,False,YES,YES,NO,NO,NO,NO,NO,120
3,2019,RESIDENTIAL,NEW YORK CITY REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,UNKNOWN,OTHER,OTHER LIVING SITUATION,...,NO,False,YES,NO,NO,NO,NO,NO,NO,114
4,2019,OUTPATIENT,WESTERN REGION,ADULT,FEMALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,PRIVATE RESIDENCE,...,NO,False,YES,YES,NO,NO,NO,NO,NO,142


In [4]:
#List the columns name
dataset.columns

Index(['Survey Year', 'Program Category', 'Region Served', 'Age Group', 'Sex',
       'Transgender', 'Sexual Orientation', 'Hispanic Ethnicity', 'Race',
       'Living Situation', 'Household Composition', 'Preferred Language',
       'Religious Preference', 'Veteran Status', 'Employment Status',
       'Number Of Hours Worked Each Week', 'Education Status',
       'Special Education Services', 'Mental Illness',
       'Intellectual Disability', 'Autism Spectrum',
       'Other Developmental Disability', 'Alcohol Related Disorder',
       'Drug Substance Disorder', 'Opioid Related Disorder',
       'Mobility Impairment Disorder', 'Hearing Impairment',
       'Visual Impairment', 'Speech Impairment', 'Hyperlipidemia',
       'High Blood Pressure', 'Diabetes', 'Obesity', 'Heart Attack', 'Stroke',
       'Other Cardiac', 'Pulmonary Asthma', 'Alzheimer or Dementia',
       'Kidney Disease', 'Liver Disease', 'Endocrine Condition',
       'Neurological Condition', 'Traumatic Brain Injury', 'J

In [5]:
#Check if there is na values
print(dataset.isna().sum())

Survey Year                       0
Program Category                  0
Region Served                     0
Age Group                         0
Sex                               0
                                 ..
Private Insurance                 0
Child Health Plus Insurance       0
Other Insurance                   0
Criminal Justice Status           0
Three Digit Residence Zip Code    0
Length: 76, dtype: int64


No #NA but Total number of columns: 76 --> Too much to work with.
Let's reduce the number of features

Workflow:
1. Group the columns by their corresponding caterogies (Demographic, clinical, social & Insurrance) with respect to the target "Mental Illness"
2. Feature evaluation: check the importance of each feature with respect to their group
3. in each group/class, aggregate features that are:
    a. Highly correlated with each other (kind of redudant)
    b. And have similar relationship with the target "Mental Illness"
4. finally, build the final feature set (new dataset with fewer columns)

In [6]:
# Define feature groups: features devided into 4 groups

feature_groups = {
    #Demographic group
    "Demographic": [
        'Survey Year', 'Program Category', 'Region Served', 'Age Group', 'Sex',
        'Transgender', 'Sexual Orientation', 'Hispanic Ethnicity', 'Race',
        'Preferred Language', 'Religious Preference', 'Veteran Status',
        'Three Digit Residence Zip Code'
    ],

    #Clinical
    "Clinical": [
        # Mental
        'Mental Illness', 'Intellectual Disability', 'Autism Spectrum',
        'Other Developmental Disability', 'Alcohol Related Disorder',
        'Drug Substance Disorder', 'Opioid Related Disorder',
        'Serious Mental Illness', 'Principal Diagnosis Class', 'Additional Diagnosis Class',
        #Physical
        'Mobility Impairment Disorder', 'Hearing Impairment', 'Visual Impairment',
        'Speech Impairment', 'Hyperlipidemia', 'High Blood Pressure', 'Diabetes',
        'Obesity', 'Heart Attack', 'Stroke', 'Other Cardiac', 'Pulmonary Asthma',
        'Alzheimer or Dementia', 'Kidney Disease', 'Liver Disease',
        'Endocrine Condition', 'Neurological Condition', 'Traumatic Brain Injury',
        'Joint Disease', 'Cancer', 'Other Chronic Med Condition',
        'No Chronic Med Condition', 'Unknown Chronic Med Condition',
        #Substance use
        'Cannabis Recreational Use', 'Cannabis Medicinal Use', 'Smokes',
        'Received Smoking Medication', 'Received Smoking Counseling',
        'Alcohol 12m Service', 'Opioid 12m Service', 'Drug/Substance 12m Service'
    ],

    # Social
    "Social": [
        'Living Situation', 'Household Composition', 'Employment Status',
        'Number Of Hours Worked Each Week', 'Education Status',
        'Special Education Services', 'Criminal Justice Status',
        'SSI Cash Assistance', 'SSDI Cash Assistance', 'Veterans Disability Benefits',
        'Veterans Cash Assistance', 'Public Assistance Cash Program',
        'Other Cash Benefits'
    ],

    # Insurance
    "Insurance": [
        'Medicaid and Medicare Insurance', 'No Insurance',
        'Unknown Insurance Coverage', 'Medicaid Insurance',
        'Medicaid Managed Insurance', 'Medicare Insurance',
        'Private Insurance', 'Child Health Plus Insurance', 'Other Insurance'
    ]
}



In [7]:
data_group={}
for group, features in feature_groups.items():
    data_group[group] = dataset[features]

Demographic = data_group["Demographic"]
Clinical = data_group["Clinical"]
Social = data_group['Social']
Insurance = data_group["Insurance"]
target = pd.DataFrame({"Mental Illness": dataset["Mental Illness"]})

# EDA Demographic Group

In [8]:
Demographic.head()

,Survey Year,Program Category,Region Served,Age Group,Sex,Transgender,Sexual Orientation,Hispanic Ethnicity,Race,Preferred Language,Religious Preference,Veteran Status,Three Digit Residence Zip Code
0,2019,OUTPATIENT,LONG ISLAND REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"YES, HISPANIC/LATINO",WHITE ONLY,ENGLISH,DATA NOT AVAILABLE,NO,117
1,2019,OUTPATIENT,NEW YORK CITY REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,ENGLISH,I BELONG TO A FORMAL RELIGIOUS GROUP,NO,100
2,2019,OUTPATIENT,HUDSON RIVER REGION,ADULT,MALE,"NO, NOT TRANSGENDER",LESBIAN OR GAY,"NO, NOT HISPANIC/LATINO",WHITE ONLY,ENGLISH,I BELONG TO A FORMAL RELIGIOUS GROUP,NO,120
3,2019,RESIDENTIAL,NEW YORK CITY REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,UNKNOWN,OTHER,ENGLISH,DATA NOT AVAILABLE,NO,114
4,2019,OUTPATIENT,WESTERN REGION,ADULT,FEMALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,ENGLISH,"I CONSIDER MYSELF SPIRITUAL, BUT NOT RELIGIOUS",NO,142


Features behavior with respect to the target

I will be using the frequency tables to evaluate how the features in the Demographic Group behave. This will allow us to see if their is any relationship between features.

Important: research on frequency tables

https://heartbeat.comet.ml/exploratory-data-analysis-eda-for-categorical-data-870b37a79b65

In [11]:
# new demographic group dataset (I add the target)
new_dem_grp = Demographic.copy()
new_dem_grp["Mental Illness"] = target["Mental Illness"].values
new_dem_grp.head()

,Survey Year,Program Category,Region Served,Age Group,Sex,Transgender,Sexual Orientation,Hispanic Ethnicity,Race,Preferred Language,Religious Preference,Veteran Status,Three Digit Residence Zip Code,Mental Illness
0,2019,OUTPATIENT,LONG ISLAND REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"YES, HISPANIC/LATINO",WHITE ONLY,ENGLISH,DATA NOT AVAILABLE,NO,117,YES
1,2019,OUTPATIENT,NEW YORK CITY REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,ENGLISH,I BELONG TO A FORMAL RELIGIOUS GROUP,NO,100,YES
2,2019,OUTPATIENT,HUDSON RIVER REGION,ADULT,MALE,"NO, NOT TRANSGENDER",LESBIAN OR GAY,"NO, NOT HISPANIC/LATINO",WHITE ONLY,ENGLISH,I BELONG TO A FORMAL RELIGIOUS GROUP,NO,120,YES
3,2019,RESIDENTIAL,NEW YORK CITY REGION,ADULT,MALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,UNKNOWN,OTHER,ENGLISH,DATA NOT AVAILABLE,NO,114,YES
4,2019,OUTPATIENT,WESTERN REGION,ADULT,FEMALE,"NO, NOT TRANSGENDER",STRAIGHT OR HETEROSEXUAL,"NO, NOT HISPANIC/LATINO",WHITE ONLY,ENGLISH,"I CONSIDER MYSELF SPIRITUAL, BUT NOT RELIGIOUS",NO,142,YES


In [17]:
# frequency_table  function
def frequency_table(data: pd.DataFrame, target_col: str, feature_cols):
    freq_tab = pd.crosstab(index = data[target_col], columns = data[feature_cols], margins = True)
    rel_table = round(freq_tab/freq_tab.loc["All"], 2)
    return freq_tab, rel_table

Univariate analysis

In [19]:
#analysis Program Category column
#Survey year column to be dropped

for col in new_dem_grp.columns[1:-2]:
    freq, rel = frequency_table(new_dem_grp, "Mental Illness", col)
    #print("Two-way frequency table")
    #print(freq)
    print("Two-way relative frequency table")
    print(rel)
    print("------" * 15)
    print()
    print()


Two-way relative frequency table
Program Category  EMERGENCY  INPATIENT  OUTPATIENT  RESIDENTIAL  SUPPORT   All
Mental Illness                                                                
NO                     0.07       0.01        0.03         0.01     0.07  0.03
UNKNOWN                0.05       0.00        0.01         0.00     0.03  0.01
YES                    0.88       0.99        0.97         0.99     0.90  0.96
All                    1.00       1.00        1.00         1.00     1.00  1.00
------------------------------------------------------------------------------------------


Two-way relative frequency table
Region Served   CENTRAL NY REGION  HUDSON RIVER REGION  LONG ISLAND REGION  \
Mental Illness                                                               
NO                           0.02                 0.03                0.05   
UNKNOWN                      0.01                 0.01                0.00   
YES                          0.97                 0.96 

It can be seen that all the features in the demographic group have almost the same behavior with respect to the target "Mental Illness"
So, we can aggregate features highly correlated and/or are kind of redudant. then use Chi-square test if we need to further reduce the number of features by selecting the ones with mos effect size

Region served and Three digit columns can be replace by Living area with values like [urban area, rural area, unknown]